# This is a source code that use to scrape data from duckduckgo to train "Is that a Supra?!" model

This code is based on AI Builders code on fantastic datasets


*  Code: https://github.com/ai-builders/curriculum/blob/main/notebooks/02_fantastic_datasets.ipynb
*   Video: https://youtu.be/1djko0F3auA


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#fastai
!pip install -q fastbook

# how to selenium on colab [Chalach Monkhontirapat](https://medium.com/equinox-blog/%E0%B8%81%E0%B8%B2%E0%B8%A3%E0%B9%83%E0%B8%8A%E0%B9%89-selenium-%E0%B8%9A%E0%B8%99-google-colaboratory-984739ed44e5)
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

#ctgan
!pip install -q sdv

#review generation
!pip install -q transformers

#reset runtime to make sure that all the packages work fine
exit()

     |████████████████████████████████| 719 kB 29.0 MB/s 
     |████████████████████████████████| 1.2 MB 35.4 MB/s 
     |████████████████████████████████| 4.2 MB 37.5 MB/s 
     |████████████████████████████████| 197 kB 56.9 MB/s 
     |████████████████████████████████| 346 kB 67.6 MB/s 
     |████████████████████████████████| 59 kB 7.2 MB/s 
     |████████████████████████████████| 140 kB 56.5 MB/s 
     |████████████████████████████████| 212 kB 58.7 MB/s 
     |████████████████████████████████| 1.1 MB 52.4 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
     |████████████████████████████████| 86 kB 6.3 MB/s 
     |████████████████████████████████| 596 kB 59.4 MB/s 
     |████████████████████████████████| 127 kB 60.6 MB/s 
     |████████████████████████████████| 271 kB 60.1 MB/s 
     |████████████████████████████████| 144 kB 60.3 MB/s 
     |████████████████████████████████| 94 kB 2.4 MB/s 
     |████████████████████████████████| 6.6 MB 47.0 MB/s 
ERROR: pip's dependenc

In [ ]:
from fastbook import *
from fastai.vision.widgets import *
from tqdm.auto import tqdm

#download from duckduckgo
def search_images_ddg(key, max_n=200):
     """Search for 'key' with DuckDuckGo and return a unique urls of 'max_n' images
        (Adopted from https://github.com/deepanprabhu/duckduckgo-images-api)
     """
     url        = 'https://duckduckgo.com/'
     params     = {'q':key}
     res        = requests.post(url,data=params)
     searchObj  = re.search(r'vqd=([\d-]+)\&',res.text)
     if not searchObj: print('Token Parsing Failed !'); return
     requestUrl = url + 'i.js'
     headers    = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:71.0) Gecko/20100101 Firefox/71.0'}
     params     = (('l','us-en'),('o','json'),('q',key),('vqd',searchObj.group(1)),('f',',,,'),('p','1'),('v7exp','a'))
     urls       = []
     while True:
         try:
             res  = requests.get(requestUrl,headers=headers,params=params)
             data = json.loads(res.text)
             for obj in data['results']:
                 urls.append(obj['image'])
                 max_n = max_n - 1
                 if max_n < 1: return L(set(urls))     # dedupe
             if 'next' not in data: return L(set(urls))
             requestUrl = url + data['next']
         except:
             pass

In [ ]:
path = Path('/content/drive/MyDrive/Toyota_image_dataset_V2.9.1') # path to folder you want to save
labels = ['Toyota 4runner', # keyword you want to search and scrape
'Toyota alphard',
'Toyota avanza',
'Toyota avensis',
'Toyota aygo',
'Toyota camry',
'Toyota celica',
'Toyota corolla',
'Toyota corona',
'Toyota crown',
'Toyota estima',
'Toyota estios',
'Toyota fortuner',
'Toyota hiace',
'Toyota highlander',
'Toyota hilux',
'Toyota innova',
'Toyota iq',
'Toyota matrix',
'Toyota mirai',
'Toyota prius',
'Toyota rav4',
'Toyota rush',
'Toyota sequoia',
'Toyota sienna',
'Toyota soarer',
'Toyota starlet',
'Toyota supra',
'Toyota tacoma',
'Toyota tundra',
'Toyota venza',
'Toyota vios',
'Toyota yaris'
]

MAX_N = 200 # amount of images you need

if not path.exists():
    path.mkdir()
    for l in tqdm(labels):
        dest = (path/l)
        dest.mkdir(exist_ok=True)
        results = search_images_ddg(l, max_n=MAX_N)
        download_images(dest, urls=results)

  0%|          | 0/33 [00:00<?, ?it/s]